In [1]:
import pandas as pd
df= pd.read_csv("Downloads/fake-news-pair-classification-challenge/train.csv", index_col=0)
df.head(3)

,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
id,,,,,,,
0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated


In [2]:
cols = ['title1_zh', 
        'title2_zh', 
        'label']
string = df.loc[:, cols].astype(str)
string.head(3)

,title1_zh,title2_zh,label
id,,,
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated


In [3]:
import jieba.posseg as pseg

def ch_tokenizer(text):
    words = pseg.cut(text)
    return ' '.join([
        word for word, flag in words if flag != 'x'])

In [4]:
string['t1_token'] = string.loc[:, 'title1_zh'].apply(ch_tokenizer)
string['t2_token'] = string.loc[:, 'title2_zh'].apply(ch_tokenizer)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\10030\AppData\Local\Temp\jieba.cache
Loading model cost 0.752 seconds.
Prefix dict has been built successfully.


In [5]:
string.head(3)

,title1_zh,title2_zh,label,t1_token,t2_token
id,,,,,
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated,2017 养老保险 又 新增 两项 农村 老人 人人 可 申领 你 领到 了 吗,警方 辟谣 鸟巢 大会 每人 领 5 万 仍 有 老人 坚持 进京
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated,你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港,深圳 GDP 首 超 香港 深圳 统计局 辟谣 只是 差距 在 缩小
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated,你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港,GDP 首 超 香港 深圳 澄清 还 差 一点点


In [6]:
import keras

tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)

In [7]:
corpus1 = string.t1_token
corpus2 = string.t2_token
corpus = pd.concat([
    corpus1, corpus2])

tokenizer.fit_on_texts(corpus)

In [8]:
X1 = tokenizer.texts_to_sequences(corpus1)
X2 = tokenizer.texts_to_sequences(corpus2)

In [9]:
X1[:1]

[[217, 1268, 32, 1178, 5967, 25, 489, 2877, 116, 5559, 4, 1850, 2, 13]]

In [10]:
X1_train = keras.preprocessing.sequence.pad_sequences(X1,maxlen=15)

X2_train = keras.preprocessing.sequence.pad_sequences(X2,maxlen=15)

In [11]:
X1_train[:5]

array([[   0,  217, 1268,   32, 1178, 5967,   25,  489, 2877,  116, 5559,
           4, 1850,    2,   13],
       [2558,    4,  166,   34,   17,   47, 5150,   63,   15,  678, 4502,
        3211,   23,  284, 1181],
       [2558,    4,  166,   34,   17,   47, 5150,   63,   15,  678, 4502,
        3211,   23,  284, 1181],
       [2558,    4,  166,   34,   17,   47, 5150,   63,   15,  678, 4502,
        3211,   23,  284, 1181],
       [   0,    0,    0,    0,    0,    0,   31,  320, 3372, 3062,    1,
          95,   98, 3372, 3062]])

In [87]:
string.label[:20]

id
0     unrelated
3     unrelated
1     unrelated
2     unrelated
9        agreed
4     unrelated
6     unrelated
5     unrelated
7     unrelated
8        agreed
11       agreed
10       agreed
13       agreed
12       agreed
15    unrelated
17    disagreed
14    unrelated
16    unrelated
18    unrelated
19    unrelated
Name: label, dtype: object

In [88]:
import numpy as np
index = {
    'unrelated': 0, 
    'agreed': 1, 
    'disagreed': 2
}

Y = string.label.apply(lambda x: index[x])
Y = np.asarray(Y).astype('float32')

Y_train = keras.utils.to_categorical(Y)
Y_train[:20]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [14]:
from sklearn.model_selection import train_test_split

X1_train, X1_val, X2_train, X2_val, Y_train, Y_val = train_test_split(X1_train, X2_train, Y_train,
                                                                      test_size=0.1,
                                                                      random_state=9487)
print(X1_train.shape)
print(X2_train.shape)
print(Y_train.shape)

(288496, 15)
(288496, 15)
(288496, 3)


In [15]:
print(X1_val.shape)
print(X2_val.shape)
print(Y_val.shape)

(32056, 15)
(32056, 15)
(32056, 3)


In [16]:
from keras import Input
from keras.layers import Embedding, LSTM, concatenate, Dense
from keras.models import Model

In [17]:
#Input Layer
first_input = Input(shape=(15, ),dtype='int32')
second_input = Input(shape=(15, ),dtype='int32')

In [45]:
#Enbedding Layer
embedding_layer = Embedding(10000, 256)

first_embedded = embedding_layer(first_input)

second_embedded = embedding_layer(second_input)

In [46]:
#Siamese LSTM Layer
siamese_lstm = LSTM(128)
first_output = siamese_lstm(first_embedded)
second_output = siamese_lstm(second_embedded)

In [47]:
#Concatenate Layer
merged = concatenate(
    [first_output, second_output], 
    axis=-1)

In [48]:
#Fully Connected Layer
dense =  Dense(
    units= 3, 
    activation='softmax')
prediction = dense(merged)

In [49]:
model = Model(
    inputs=[first_input, second_input], 
    outputs=prediction)

In [50]:
model.summary()

Model: "functional_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 15, 256)      2560000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          197120      embedding_1[0][0]     

In [51]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [52]:
train = model.fit(
    x=[X1_train, X2_train], 
    y=Y_train,
    batch_size= 512,
    epochs=10,
    validation_data=(
        [X1_val, X2_val], 
        Y_val
    ),
    shuffle=True
)

Epoch 1/10
564/564 [==============================] - 101s 179ms/step - loss: 0.4621 - accuracy: 0.7873 - val_loss: 0.3886 - val_accuracy: 0.8244
Epoch 2/10
564/564 [==============================] - 93s 165ms/step - loss: 0.3584 - accuracy: 0.8382 - val_loss: 0.3684 - val_accuracy: 0.8337
Epoch 3/10
564/564 [==============================] - 94s 166ms/step - loss: 0.3208 - accuracy: 0.8574 - val_loss: 0.3677 - val_accuracy: 0.8343
Epoch 4/10
564/564 [==============================] - 92s 163ms/step - loss: 0.2917 - accuracy: 0.8717 - val_loss: 0.3620 - val_accuracy: 0.8411
Epoch 5/10
564/564 [==============================] - 96s 170ms/step - loss: 0.2674 - accuracy: 0.8833 - val_loss: 0.3693 - val_accuracy: 0.8442
Epoch 6/10
564/564 [==============================] - 92s 164ms/step - loss: 0.2439 - accuracy: 0.8941 - val_loss: 0.3731 - val_accuracy: 0.8469
Epoch 7/10
564/564 [==============================] - 95s 169ms/step - loss: 0.2230 - accuracy: 0.9036 - val_loss: 0.3870 - val_a

In [56]:
import pandas as pd
testdf = pd.read_csv(
    "Downloads/fake-news-pair-classification-challenge/test.csv", index_col=0)

In [58]:
#Word Segmentation
cols = ['title1_zh', 
        'title2_zh']
test = testdf.loc[:, cols].astype(str)
test['title1_token'] = test.loc[:, 'title1_zh'].apply(ch_tokenizer)
test['title2_token'] = test.loc[:, 'title2_zh'].apply(ch_tokenizer)

#convert the text to the list of values
X1_test = tokenizer.texts_to_sequences(test.title1_token)
X2_test = tokenizer.texts_to_sequences(test.title2_token)

#zero padding
X1_test = keras.preprocessing.sequence.pad_sequences(X1_test,maxlen=15)
X2_test = keras.preprocessing.sequence.pad_sequences(X2_test,maxlen=15)

#predict by TRAINED model
prediction = model.predict(
    [X1_test, X2_test])

In [86]:
prediction[:8]

array([[9.9832112e-01, 9.4468341e-08, 1.6787463e-03],
       [9.9996686e-01, 4.8061061e-06, 2.8400182e-05],
       [9.9932313e-01, 6.4231828e-04, 3.4583667e-05],
       [9.9989581e-01, 6.7705673e-06, 9.7389995e-05],
       [9.9979550e-01, 1.1927923e-05, 1.9255393e-04],
       [9.3867499e-01, 6.1307047e-02, 1.7949895e-05],
       [9.9997067e-01, 5.2129108e-06, 2.4117866e-05],
       [2.2783574e-01, 7.7061427e-01, 1.5498924e-03]], dtype=float32)

In [90]:
label = {v: k for k, v in index.items()}

test['Category'] = [label[lx] for lx in np.argmax(prediction, axis=1)]

result = test.loc[:, ['Category']].reset_index()

result.columns = ['Id', 'Category']
result.head(50)

,Id,Category
0,321187,unrelated
1,321190,unrelated
2,321189,unrelated
3,321193,unrelated
4,321191,unrelated
5,321194,unrelated
6,321192,unrelated
7,321197,agreed
8,321195,agreed
9,321199,unrelated
